In [47]:
library(dplyr)
library(caret)
library(MASS)

# Дискриминантный анализ 
## Фахртдинов Тимур

Построим процедуру линейной классификации по восьми независимым признакам, прогнозирующих смерть младенцев.

In [64]:
data <-read.csv("baseMA.csv", sep = ";")
data <- data.frame(Outcome = data$Outcome, Diuresis1 = data$Diuresis1, Weight1 = data$Weight1, 
                   Duration.in.hosp = data$Duration.in.hosp, PA1 = data$PA1, 
                   RBC1 = data$RBC1, PS1 = data$PS1, DF1 = data$DF1, Sat1 = data$Sat1)
data <- na.omit(data)

Найдем коэффициенты $\alpha_{i}$ дискриминантной функции вида

$$z = \alpha_{1}x_{1}+\dots+\alpha_{p}x_{p} = \alpha^{T}X$$

In [65]:
W1 = data[data$Outcome == 1,][-1]
W2 = data[data$Outcome == 0,][-1]

n1 = length(data$Outcome[data$Outcome == 1])
n2 = length(data$Outcome[data$Outcome == 0])

X1 = 1 / n1 * apply(W1, 2, sum) 
X2 = 1 / n2 * apply(W2, 2, sum)

S = 1 / (n1 + n2 - 2) * ((n1 - 1) * cov(W1) + (n2 - 1) * cov(W2))
alpha = solve(S, X1 - X2)
alpha

Diuresis1          Weight1 Duration.in.hosp              PA1 
   -0.1296733984    -0.0001718258     0.0127597272     0.0347032828 
            RBC1              PS1              DF1             Sat1 
    0.7096164619    -0.0101660828    -0.0144560043     0.1064910245

Проверим встроенной функцией:

In [66]:
m = as.matrix(data[-1])
m = as.vector(m %*% alpha)

l <- lda(data$Outcome ~., data)
print(l)

Call:
lda(data$Outcome ~ ., data = data)

Prior probabilities of groups:
       0        1 
0.195122 0.804878 

Group means:
  Diuresis1  Weight1 Duration.in.hosp      PA1     RBC1      PS1      DF1
0  6.039167 1072.917         61.95833 45.70833 3.657083 155.3750 34.37500
1  5.708788 1071.263         90.19192 50.36364 4.066869 154.9899 33.25253
      Sat1
0 93.83333
1 95.00000

Coefficients of linear discriminants:
                           LD1
Diuresis1        -0.1296666476
Weight1          -0.0001718168
Duration.in.hosp  0.0127590629
PA1               0.0347014762
RBC1              0.7095795194
PS1              -0.0101655536
DF1              -0.0144552517
Sat1              0.1064854806


Найдем граничное значение дискриминантной функции:

In [81]:
q1 = n1 / (n1 + n2)
q2 = 1 - q1

z.k = (t(alpha) %*% X1 + t(alpha) %*% X2)/2 + log(q2/q1)
z.k = as.numeric(z.k)

z.k

[1] 11.00337

Построим матрицу классификации:

In [82]:
a = b = c = d = 0
for (i in 1:length(m)){
  if (m[i] >= z.k  & data$Outcome[i] == 1) a = a + 1
  if (m[i] < z.k & data$Outcome[i] == 1) b = b + 1
  if (m[i] >= z.k & data$Outcome[i] == 0) c = c + 1
  if (m[i] < z.k & data$Outcome[i] == 0) d = d + 1
}
dm <- rbind(c(a/(a+b), a, b), c(d/(c + d), c, d), c((a/(a+b) + d/(c + d))/2, a + c, b + d))

colnames(dm)<-c("Процент", "W1", "W2")
rownames(dm)<-c('W1','W2', "Всего")

dm
table(data$Outcome, predict(l)$class)

,Процент,W1,W2
W1,1.0000000,99,0
W2,0.2083333,19,5
Всего,0.6041667,118,5


   
     0  1
  0  5 19
  1  0 99

Процент правильной классификации для выживших младенцев:

In [71]:
1 -  b / (a + b)

[1] 1

Процент правильной классификации для умерших младенцев:


In [72]:
1 - c / (c + d)

[1] 0.2083333